In [1]:
import qgrid
import pandas as pd
import time
import warnings

def translate_stringy_list(stringy_list):
    #pd.read_csv imports listboundvariables dtype as a string, we want it in list form
    final_list = []
    stringy_list = stringy_list.split(',')
    for var in stringy_list:
        final_list.append(var.translate({ord(c): None for c in '[ ]\,\''}))
    return final_list

def save_image_log(event, qgrid_widget):
    df_display = qgrid_widget.get_changed_df()
    qgrid_widget.close()
    df = pd.read_csv(filename)
    for idx in range(len(df_display)):
        for column in df_display.columns:
            df.loc[idx, column] = df_display.loc[idx, column]
    df.to_csv(filename, index = False)
    qgrid_widget = edit_image_log()
    qgrid_widget.on('cell_edited',save_image_log)
    display(qgrid_widget)

def edit_image_log(new_columns = {}):
    #new_columns accepts a dictionary of {column_name: default value}, e.g. {'filter_in': False, 'OD':4}
    #set default value of string dtypes as '[space]' for easier parsing
    
#     try: #save the edited log if it was opened and edited
#         save_image_log(filename)
#     except:
#         pass
    df = pd.read_csv(filename, dtype={'badshot': 'bool'})
    for column_name in new_columns:
        if column_name not in df.columns:
            df[column_name] = new_columns[column_name]
        else:
            warnings.warn('column \'' + column_name + '\' already exists')
    
    list_bound_vars = translate_stringy_list(df['ListBoundVariables'][0])
    if 'editable_variables' in df.columns:
        editable_vars = translate_stringy_list(df['editable_variables'][0])
        editable_vars = editable_vars + [column_name for column_name in new_columns]
    else:
        editable_vars = [column_name for column_name in new_columns]
    if len(editable_vars) > 0:
        df['editable_variables'] = str(editable_vars)
    
#     display_vars = ['badshot', 'notes'] + list_bound_vars + ['fake analysis'] + editable_vars + ['runtime']
    display_vars = ['badshot', 'notes'] + list_bound_vars + editable_vars + ['runtime', 'analyzed_variables']
    df_display = df[display_vars]
    qgrid_widget = qgrid.show_grid(df_display, grid_options = {'fullWidthRows' : False})
    return qgrid_widget

In [3]:
global filename
filename = input(r'enter run directory, e.g. 07\200729\foo\ for run foo on Jul 29,2020:')#'07\200729\foo\'
filename = filename + 'image_log.csv'
qgrid_widget = edit_image_log()
qgrid_widget.on('cell_edited',save_image_log)
display(qgrid_widget)

enter run directory, e.g. 07\200729\foo\ for run foo on Jul 29,2020:07\200729\foo\


QgridWidget(grid_options={'fullWidthRows': False, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defa…